In [2]:
import os
import spatialite

from sqlalchemy import create_engine
from sqlalchemy.event import listen

from langchain import LLMChain, OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [3]:
model = "gpt-4"
temperature = 0.7

In [4]:
chat = ChatOpenAI(
    model_name=model,
    temperature=temperature,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)

In [5]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    """
        You are a helpful assistant with experience with SQL,
        sqlite3 and spatialite relational databases and query languages.'

        Give only the query commands as answers.
    """
)

human_message_prompt = HumanMessagePromptTemplate.from_template(
    """
        How to query {query} from a sqlite3 database?
    """
)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [6]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
res = chain.run(query="names of all tables")
res

"SELECT name FROM sqlite_master WHERE type='table';"

In [63]:
def load_spatialite(dbapi_conn, _):
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension('/usr/lib/x86_64-linux-gnu/mod_spatialite.so')

engine = create_engine("sqlite+pysqlite:///workspaces/forest-analyzer/data/gis.db")
listen(engine, "connect", load_spatialite)
engine.connect()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [55]:
db = SQLDatabase(engine=engine)
db_chain = SQLDatabaseChain(llm=chat, database=db, verbose=True)

In [45]:
db_chain.run("Names of all tables")



> Entering new SQLDatabaseChain chain...
Names of all tables 
SQLQuery:SELECT name FROM sqlite_master WHERE type='table';
SQLResult: []
Answer:There are no tables in the database.
> Finished chain.


'There are no tables in the database.'

In [6]:
conn = spatialite.connect('../data/MV_K3242F.gpkg')                                
cursor = conn.cursor()

cursor.execute(res)
cursor.fetchall()

[('gpkg_spatial_ref_sys',),
 ('gpkg_geometry_columns',),
 ('gpkg_extensions',),
 ('gpkg_contents',),
 ('treestratum',),
 ('sqlite_sequence',),
 ('treestandsummary',),
 ('treestand',),
 ('stand',),
 ('specification',),
 ('specialfeature',),
 ('restriction',),
 ('operation',),
 ('datasource',),
 ('assortment',),
 ('rtree_stand_geometry',),
 ('rtree_stand_geometry_node',),
 ('rtree_stand_geometry_rowid',),
 ('rtree_stand_geometry_parent',)]